In [1]:
import os

api_key = os.getenv("DEEPSEEK_API_KEY")

In [ ]:
import autogen

config_list_deepseek_reasoner = [
    {
        "model": "deepseek-reasoner",
        "base_url": "https://api.deepseek.com/v1",
        "api_key": api_key,
        "api_type": "deepseek",
        "tags": ["deepseek"],
    }
]
config_list_openai_4o_mini = [
    {
        "model": "gpt-4o-mini",
        "api_key": os.getenv("OPENAI_API_KEY"),
    }
]
user_proxy = autogen.UserProxyAgent(
    "user_proxy",
    human_input_mode="NEVER",
    code_execution_config={"work_dir": "coding", "use_docker": False},
)

supervisor = autogen.AssistantAgent(
    "supervisor",
    llm_config={
        "config_list": config_list_deepseek_reasoner,
    },
)


assistant = autogen.AssistantAgent(
    "assistant",
    llm_config={
        "config_list": config_list_deepseek_reasoner,
    },
)


groupchat = autogen.GroupChat(
    agents=[user_proxy, supervisor, assistant],
    messages=["A group chat"],
    max_round=5,
)

manager = autogen.GroupChatManager(
    groupchat=groupchat,
    llm_config={
        # Error is expected using deepseek_reasoner as manager
        # "config_list": config_list_deepseek_reasoner,
        #############
        # raise self._make_status_error_from_response(err.response) from None
        # openai.BadRequestError: Error code: 400 -
        # {'error': {'message': 'The last message of deepseek-reasoner must be a user message,
        # or an assistant message with prefix mode on (refer to https://api-docs.deepseek.com/guides/chat_prefix_completion).',
        # 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_request_error'}}
        #############
        # Another expected using openai_4o_mini as manager
        "config_list": config_list_openai_4o_mini,
        #############
        # Error code: 400 - {'error': {'message': 'deepseek-reasoner does not support successive
        # user or assistant messages (messages[1] and messages[2] in your input).
        # You should interleave the user/assistant messages in the message sequence.',
        # 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_request_error'}}
        #############
    },
)


# Group Chat
user_proxy.initiate_chat(
    manager,
    message="""

    Save a chart of NVDA and TESLA stock price change YTD.

    Start the chat with assistant.

    And ask supervisor to check the work from assistant.

    """,
)

In [ ]:
# import autogen
# from autogen.coding import LocalCommandLineCodeExecutor


# class DeepSeekReasonerAssistant(autogen.AssistantAgent):
#     def __init__(self, name, **kwargs):
#         super().__init__(name, **kwargs)

#     # def generate_reply(self, messages=None, sender=None, config=None):
#     #     print(f"\n=== {self.name} Processing Messages ===")
#     #     if messages is None:
#     #         messages = self._oai_messages[sender]
#     #     print(f"Messages: {messages}")

#     #     # Get messages from chat history if available
#     #     if hasattr(self, "chat_messages") and sender in self.chat_messages:
#     #         chat_history = self.chat_messages[sender]
#     #     else:
#     #         chat_history = messages if messages else []
#     #     print(f"Chat History: {chat_history}")

#     #     if chat_history:
#     #         # Create new message sequence
#     #         new_messages = []

#     #         # Add system message if it exists
#     #         if hasattr(self, "system_message"):
#     #             new_messages.append({"role": "system", "content": self.system_message})

#     #         # Add original messages ensuring alternation
#     #         for msg in chat_history:
#     #             if len(new_messages) == 0 or msg["role"] != new_messages[-1]["role"]:
#     #                 new_messages.append(msg)

#     #         # Check if last message was from assistant
#     #         if new_messages and new_messages[-1]["role"] == "assistant":
#     #             new_messages.append({"role": "user", "content": "continue"})
#     #         print(f"New Messages: {new_messages}")

#     #         # return super().generate_reply(messages=new_messages, sender=sender, config=config)
#     #     return super().generate_reply(messages=messages, sender=sender, config=config)


# # Create agents using the custom class
# assistant = DeepSeekReasonerAssistant(
#     "assistant",
#     system_message="You are a coding assistant. Write code for the given task.",
#     llm_config={
#         "config_list": config_list_deepseek_reasoner,
#     },
# )

# supervisor = DeepSeekReasonerAssistant(
#     "supervisor",
#     system_message="""You are a code reviewer. Review code for:
#     1. Correctness
#     2. Error handling
#     3. Performance
#     4. Best practices""",
#     llm_config={
#         "config_list": config_list_deepseek_reasoner,
#     },
# )


# user_proxy = autogen.UserProxyAgent(
#     "user_proxy",
#     human_input_mode="NEVER",
#     code_execution_config={
#         "executor": LocalCommandLineCodeExecutor(work_dir="coding"),
#     },
#     is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
# )

# # Create group chat
# groupchat = autogen.GroupChat(agents=[user_proxy, assistant, supervisor], messages=[], max_round=5)

# # Manager with chat model
# manager = autogen.GroupChatManager(
#     groupchat=groupchat,
#     system_message="""You are the coordinator. Follow this sequence:
#     1. When user asks a question, select assistant to write code
#     2. After assistant provides code, select supervisor to review it
#     3. If the problem has been resolved respond with TERMINATE""",
#     llm_config={
#         "config_list": config_list_openai_4o_mini,
#     },
#     is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
# )

# print("Starting chat...")
# user_proxy.initiate_chat(manager, message="Give me some info about the stock market.")

In [ ]:
{
    "messages": [
        {
            "content": "You are a code reviewer. Review code for:\n    1. Correctness\n    2. Error handling\n    3. Performance\n    4. Best practices",
            "role": "system",
        },
        {"content": "Create a chart showing NVDA and TESLA stock prices.", "name": "user_proxy", "role": "user"},
        {
            "content": "To create a chart comparing NVDA and TESLA stock prices, you can use Python with libraries like `yfinance` (to fetch stock data) and `matplotlib` (for plotting). Here's a complete example:\n\n```python\n# Install required libraries if needed\n# !pip install yfinance matplotlib\n\nimport yfinance as yf\nimport matplotlib.pyplot as plt\n\n# Define the tickers and time period\ntickers = ['NVDA', 'TSLA']\nstart_date = '2023-01-01'\nend_date = '2024-01-01'\n\n# Download historical data\ndata = yf.download(tickers, start=start_date, end=end_date)['Adj Close']\n\n# Create plot\nplt.figure(figsize=(12, 6))\nplt.plot(data['NVDA'], label='NVIDIA (NVDA)', color='#76b900')  # NVIDIA green\nplt.plot(data['TSLA'], label='Tesla (TSLA)', color='#cc0000')    # Tesla red\n\n# Formatting\nplt.title('NVDA vs TESLA Stock Prices (2023)', fontsize=14)\nplt.xlabel('Date', fontsize=12)\nplt.ylabel('Adjusted Close Price (USD)', fontsize=12)\nplt.legend()\nplt.grid(True, which='both', linestyle='--', linewidth=0.5)\nplt.xticks(rotation=45)\nplt.tight_layout()\n\n# Show plot\nplt.show()\n```\n\nThis code will:\n1. Fetch adjusted closing prices for both stocks from Yahoo Finance\n2. Create a line chart comparing their performance\n3. Use official brand colors (NVIDIA green and Tesla red)\n4. Include proper labels and formatting\n\nThe chart will show the stock price trends from January 2023 to January 2024. You can modify the `start_date` and `end_date` variables to adjust the time period.\n\nMake sure you have an internet connection as the script needs to download stock data from Yahoo Finance. The first time you run it, you might need to install the required libraries using pip.",
            "name": "assistant",
            "role": "user",
        },
        {
            "content": "exitcode: 1 (execution failed)\nCode output: Traceback (most recent call last):\n  File \"/Users/robert/projects/ag2/notebook/coding/Install required libraries if needed\", line 4, in <module>\n    import yfinance as yf\nModuleNotFoundError: No module named 'yfinance'\n",
            "name": "user_proxy",
            "role": "user",
        },
    ],
    "model": "deepseek-reasoner",
    "stream": False,
}